In [1]:
%matplotlib inline
import os
os.environ['PY3_PROD'] = '1'
%load_ext autoreload
%autoreload 2
os.system('kinit')

1

In [2]:
import numpy as np
import pandas as pd
from typing import Union, List
import datetime
from pycmqlib3.utility import misc

from pycmqlib3.utility.dbaccess import *
from pycmqlib3.utility import dataseries
from pycmqlib3.analytics.tstool import *
from pycmqlib3.analytics.btmetrics import *

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
#pd.set_option('display.width', 1000)

In [ ]:
update_contract_list_table(datetime.date(2023,1,10))

# load eod mark from exchange

In [ ]:
from misc_scripts.aks_data_update import update_hist_fut_daily

update_hist_fut_daily(start_date=datetime.date(2022,12,22),
                          end_date=datetime.date(2023,1,6),
                          exchanges=['GFEX'],
                          flavor='mysql',
                          fut_table='fut_daily')

# check factor data

In [ ]:
prod_list = ['rb', 'hc', 'i', 'j', 'jm']
roll_label = 'CAL_30b'
freq = 's1'
db_table = 'fut_fact_data'
fact_list = ["ryield_1_nmb_120", "basmom_100_nma_120", "basmom_170_nma_120",
            "ryield_1_nma_20", "ryield_1_nma_110"]
start_date = datetime.date(2022,10,1)
end_date = datetime.date(2022,12,30)
df = load_factor_data(prod_list, 
                      factor_list = fact_list,
                      roll_label = roll_label,
                      start = start_date,
                      end = end_date,
                      freq = freq,
                      db_table = db_table)
print(df)

In [3]:
import json
import datetime
import json
from pycmqlib3.utility.misc import day_shift, CHN_Holidays, sign, is_workday, inst2product
import pycmqlib3.analytics.data_handler as dh
from misc_scripts.factor_data_update import update_factor_data, generate_daily_position

In [19]:
scenarios_all = [
    ('tscarry', 'ryieldnmb', 2.8, 1, 120, 1, (None, {}, ''), [0.0, 0.0]),

    ('tscarry', 'ryieldnmb', 1.0, 1, 122, 1, (None, {}, ''), [0.0, 0.0]),
    ('tscarry', 'ryieldqtl', 0.8, 1, 20, 1, (None, {}, ''), [0.0, 0.0]),
    ('tscarry', 'ryieldqtl', 0.8, 1, 60, 1, (None, {}, ''), [0.0, 0.0]),
    ('tscarry', 'ryieldqtl', 0.8, 1, 244, 1, (None, {}, ''), [0.0, 0.0]),

    ('tscarry', 'basmomnma', 0.7, 100, 120, 1, (None, {}, ''), [0.0, 0.0]),
    ('tscarry', 'basmomnma', 0.5, 170, 120, 1, (None, {}, ''), [0.0, 0.0]),
    # ('tscarry', 'basmomnma', 0.2, 230, 120, 1, (None, {}, ''), [0.0, 0.0]),

    ('tscarry', 'basmomnma', 0.5, 20, 122, 1, (None, {}, ''), [0.0, 0.0]),
    ('tscarry', 'basmomnma', 0.42, 60, 122, 1, (None, {}, ''), [0.0, 0.0]),
    ('tscarry', 'basmomnma', 0.35, 120, 122, 1, (None, {}, ''), [0.0, 0.0]),
    ('tscarry', 'basmomnma', 0.35, 180, 122, 1, (None, {}, ''), [0.0, 0.0]),
    ('tscarry', 'basmomqtl', 2.0, 120, 20, 1, (None, {}, ''), [0.0, 0.0]),
    ('tscarry', 'basmomqtl', 1.8, 240, 20, 1, (None, {}, ''), [0.0, 0.0]),

    ('xscarry', 'ryieldsma', 0.6, 1, 30, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    # ('xscarry', 'ryieldsma', 0.15, 1, 110, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry', 'ryieldsma', 1.5, 1, 190, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry', 'ryieldnma', 1.5, 1, 20, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry', 'ryieldnma', 1.8, 1, 110, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    # ('xscarry', 'ryieldnma', 0.2, 1, 210, 1, (None, {}, ''), [0.0, 0.0], 0.2),

    ('xscarry-rank', 'ryieldnma', 1.4, 1, 20, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry-rank', 'ryieldnma', 1.4, 1, 122, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry-rank', 'ryieldnma', 1.4, 1, 244, 1, (None, {}, ''), [0.0, 0.0], 0.2),

    ('xscarry', 'basmomsma', 0.6, 100, 10, 5, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry', 'basmomsma', 0.6, 220, 10, 5, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry', 'basmomnma', 1.5, 80, 120, 5, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry', 'basmomnma', 1.5, 150, 120, 5, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry', 'basmomnma', 1.5, 220, 120, 5, (None, {}, ''), [0.0, 0.0], 0.2),

    ('xscarry-rank', 'basmomnma', 2.0, 20, 122, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry-rank', 'basmomnma', 2.0, 100, 122, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xscarry-rank', 'basmomnma', 2.0, 170, 122, 1, (None, {}, ''), [0.0, 0.0], 0.2),

    ('tsmom', 'hlbrk', 2.0, 10, 1, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('tsmom', 'hlbrk', 1.5, 30, 1, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('tsmom', 'hlbrk', 1.2, 240, 1, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('tsmom', 'momnma', 0.2, 10, 60, 1, (None, {}, ''), [0.0]),
    ('tsmom', 'momnma', 0.07, 220, 60, 1, (None, {}, ''), [0.0]),
    # ('tsmom', 'momxma', 0.2, 40, 30, 5, (misc.sign, {}, 'sign'), [0.0]),
    # ('tsmom', 'momxma', 0.15, 40, 80, 5, (misc.sign, {}, 'sign'), [0.0]),
    # ('tsmom', 'mixmom', 0.375, 10, 1, 10, (misc.sign, {}, 'sign'), [0.0]),
    # ('tsmom', 'mixmom', 0.3, 30, 1, 10, (misc.sign, {}, 'sign'), [0.0]),
    # ('tsmom', 'mixmom', 0.3, 220, 1, 10, (misc.sign, {}, 'sign'), [0.0]),
    # ('tsmom', 'rsixea', 0.25, 30, 40, 5, (misc.sign, {}, 'sign'), [0.0]),
    # ('tsmom', 'rsixea', 0.25, 30, 110, 5, (misc.sign, {}, 'sign'), [0.0]),
    # ('xsmom', 'mom', 0.15, 160, 1, 5, (None, {}, ''), [0.0], 0.2),

    ('tsmom', 'hlbrk', 0.5, 20, 1, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('tsmom', 'hlbrk', 0.5, 40, 1, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('tsmom', 'hlbrk', 0.5, 61, 1, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('tsmom', 'hlbrk', 0.5, 122, 1, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('tsmom', 'hlbrk', 0.5, 244, 1, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('tsmom', 'macdnma', 0.22, 8, 160, 5, (dh.response_curve, {"response": "reverting", "param": 2}, 'reverting'),
     [1.5, 10.0]),
    ('tsmom', 'macdnma', 0.20, 16, 160, 5, (dh.response_curve, {"response": "reverting", "param": 2}, 'reverting'),
     [1.5, 5.0]),
    ('tsmom', 'macdnma', 0.18, 24, 160, 5, (dh.response_curve, {"response": "reverting", "param": 2}, 'reverting'),
     [1.5, 3.34]),

    ('xsmom', 'hlbrk', 1.2, 120, 1, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xsmom', 'hlbrk', 1.2, 240, 1, 10, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xsmom', 'mom', 1.0, 20, 1, 5, (None, {}, ''), [0.0], 0.2),
    ('xsmom', 'mom', 1.0, 210, 1, 5, (None, {}, ''), [0.0], 0.2),
    ('xsmom', 'momnma', 1.0, 130, 90, 5, (None, {}, ''), [0.0], 0.2),
    ('xsmom', 'momnma', 1.0, 240, 90, 5, (None, {}, ''), [0.0], 0.2),
    ('xsmom', 'momsma', 0.8, 140, 120, 5, (None, {}, ''), [0.0], 0.2),
    ('xsmom', 'momsma', 0.8, 240, 120, 5, (None, {}, ''), [0.0], 0.2),
    # ('xsmom', 'rsiema', 0.1, 70, 60, 5, (None, {}, ''), [0.0], 0.2),
    # ('xsmom', 'rsiema', 0.1, 100, 80, 5, (None, {}, ''), [0.0], 0.2),
    # ('xsmom', 'rsiema', 0.1, 90, 10, 5, (None, {}, ''), [0.0], 0.2),
    # ('xsmom', 'macdnma', 0.1, 8, 200, 5, (dh.response_curve, {"response": "absorbing", "param": 2}, "absorbing"), [1.5, 12.5], 0.2),
    # ('xsmom', 'macdnma', 0.1, 16, 200, 5, (dh.response_curve, {"response": "absorbing", "param": 2}, "absorbing"), [1.5, 6.25], 0.2),
    # ('xsmom', 'macdnma', 0.1, 32, 200, 5, (dh.response_curve, {"response": "absorbing", "param": 2}, "absorbing"), [1.5, 3.125], 0.2),
    # ('xsmom', 'macdnma', 0.1, 64, 100, 5, (dh.response_curve, {"response": "absorbing", "param": 2}, "absorbing"), [1.5, 1.56], 0.2),

    ('xsmom-rank', 'hlbrk', 0.375, 20, 1, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xsmom-rank', 'hlbrk', 0.375, 40, 1, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xsmom-rank', 'hlbrk', 0.375, 61, 1, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xsmom-rank', 'hlbrk', 0.375, 122, 1, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xsmom-rank', 'hlbrk', 0.375, 244, 1, 1, (None, {}, ''), [0.0, 0.0], 0.2),
    ('xsmom-rank', 'momnma', 0.6, 10, 10, 1, (None, {}, ''), [0.0], 0.2),
    ('xsmom-rank', 'momnma', 0.6, 130, 120, 1, (None, {}, ''), [0.0], 0.2),
    ('xsmom-rank', 'momnma', 0.6, 240, 60, 1, (None, {}, ''), [0.0], 0.2),
]


commod_mkts = ['rb', 'hc', 'i', 'j', 'jm', 'ru', 'FG', 'cu', 'al', 'zn', 'pb', 'ni', 'sn', \
               'l', 'pp', 'v', 'TA', 'sc', 'm', 'RM', 'y', 'p', 'OI', 'a', 'c', 'CF', 'jd', \
               'AP', 'SM', 'SF', 'ss', 'CJ', 'UR', 'eb', 'eg', 'pg', 'T', 'PK', 'PF', 'lh', \
               'MA', 'SR', 'cs', 'TF', 'lu', 'fu']

In [20]:
edate=datetime.date(2023,1,20)
start_date = day_shift(edate, '-30m')

run_settings = [
    ('commod_cal', commod_mkts, scenarios_all, 'CAL_30b', 's1'),
    ('commod_hot', commod_mkts, scenarios_all, 'hot', 'd1'),
    ('commod_exp', commod_mkts, scenarios_all, 'expiry', 'd1'),
]

res = {}
for (fact_key, fact_mkts, scenarios, roll_label, freq) in run_settings:
    res[fact_key] = update_factor_data(fact_mkts, scenarios, start_date, edate, roll_rule=roll_label, freq=freq)

loading mkt = rb


C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:76: SAWarning: Can't validate argument 'replace_string'; can't locate any SQLAlchemy dialect named 'replace'
  conn.execute(table.table.insert(replace_string=""), data)


loading mkt = hc
loading mkt = i
loading mkt = j
loading mkt = jm
loading mkt = ru
loading mkt = FG
loading mkt = cu
loading mkt = al
loading mkt = zn
loading mkt = pb
loading mkt = ni
loading mkt = sn
loading mkt = l
loading mkt = pp
loading mkt = v
loading mkt = TA
loading mkt = sc
loading mkt = m
loading mkt = RM
loading mkt = y
loading mkt = p
loading mkt = OI
loading mkt = a
loading mkt = c
loading mkt = CF
loading mkt = jd
loading mkt = AP
loading mkt = SM
loading mkt = SF
loading mkt = ss
loading mkt = CJ
loading mkt = UR
loading mkt = eb
loading mkt = eg
loading mkt = pg
continuous contract stopped at pg2010 for start = 2020-09-05, expiry= 2020-09-07
loading mkt = T
loading mkt = PK
loading mkt = PF
loading mkt = lh
continuous contract stopped at lh2107 for start = 2021-05-01, expiry= 2021-06-07
loading mkt = MA
loading mkt = SR
loading mkt = cs
loading mkt = TF
loading mkt = lu
continuous contract stopped at lu2012 for start = 2020-10-01, expiry= 2020-10-26
loading mkt = fu
lo

In [10]:


posfile = '%s%s_%s.json' % (pos_loc, port_file, pos_date)

with open("C:/dev/data/new_strat.json", 'w') as f:
    json.dump(res, f, indent=4)

In [41]:
pos_chg_notification = ['PT_FACTPORT3_CAL_30b', 'PT_FACTPORT1_hot']
port_pos_config = [
    ('PT_FACTPORT3', 'C:/dev/pyktrader3/process/pt_test3/', 4600, 'CAL_30b', 's1'),
    ('PT_FACTPORT1', 'C:/dev/pyktrader3/process/pt_test3/', 4600, 'CAL_30b', 's1'),
    ('PT_FACTPORT3', 'C:/dev/pyktrader3/process/pt_test3/', 4600, 'hot', 'd1'),
    ('PT_FACTPORT1', 'C:/dev/pyktrader3/process/pt_test3/', 4600, 'hot', 'd1'),
    ('PT_FACTPORT3', 'C:/dev/pyktrader3/process/pt_test3/', 4600, 'expiry', 'd1'),
    ('PT_FACTPORT1', 'C:/dev/pyktrader3/process/pt_test3/', 4600, 'expiry', 'd1'),
]

edate=datetime.date(2023,1,31)
target_pos = {}
pos_sum = {}
for (port_name, pos_loc, pos_scaler, roll, freq) in port_pos_config:
    port_file = port_name + '_' + roll
    config_file = f'{pos_loc}settings/{port_name}.json'
    with open(config_file, 'r') as fp:
        strat_conf = json.load(fp)
    strat_args = strat_conf['config']
    assets = strat_args['assets']
    factor_repo = strat_args['factor_repo']
    product_list = []
    
    for asset_dict in assets:
        under = asset_dict["underliers"][0]
        product = inst2product(under)
        product_list.append(product)


    target_pos, pos_sum = generate_daily_position(edate, product_list, factor_repo,
                                                  roll_label=roll,
                                                  pos_scaler=pos_scaler,
                                                  freq=freq,
                                                  hist_fact_lookback=20)
    pos_date = day_shift(edate, '1b', CHN_Holidays)
    pre_date = day_shift(pos_date, '-1b', CHN_Holidays)
    pos_date = pos_date.strftime('%Y%m%d')
    pre_date = pre_date.strftime('%Y%m%d')
    posfile = '%s%s_%s.json' % (pos_loc, port_file, pos_date)
    with open(posfile, 'w') as ofile:
        json.dump(target_pos, ofile, indent=4)
    pos_sum.index.name = 'factor'
    pos_sum.to_csv('%spos_by_strat_%s_%s.csv' % (pos_loc, port_file, pos_date))
    if port_file in pos_chg_notification:
        with open('%s%s_%s.json' % (pos_loc, port_file, pre_date), 'r') as fp:
            curr_pos = json.load(fp)

        pos_df = pd.DataFrame({'cur': curr_pos, 'tgt': target_pos})

        pos_df['diff'] = pos_df['tgt'] - pos_df['cur']

        pos_update[port_file] = pos_df


In [43]:
from pycmqlib3.utility.email_tool import send_html_by_smtp
from pycmqlib3.utility.sec_bits import EMAIL_HOTMAIL


job_status={
    "fut_daily": {
        "DCE": True,
        "CFFEX": True,
        "CZCE": True,
        "SHFE": True,
        "INE": True,
        "GFEX": True
    },
    "fact_repo": {
        "commod_cal": True
    }
}
edate=datetime.date(2023,1,31)
sub = 'EOD pos and job status<%s>' % (edate.strftime('%Y.%m.%d'))
html = "<html><head></head><body><p><br>"
for key in pos_update:
    html += "Position change for %s:<br>%s" % (key, pos_update[key].to_html())
html += "Job status: %s <br>" % (json.dumps(job_status))
html +="</p></body></html>"
send_html_by_smtp(EMAIL_HOTMAIL, ['harveywu@gmail.com'], sub, html)

True

# Excel sheets updating and saving

In [8]:
import os
import time
import pandas as pd
import win32com.client as win32 

In [9]:

import win32com.client as win32
excel = win32.gencache.EnsureDispatch('Excel.Application')
outfile = 'C:/Users/harvey/OneDrive/Documents/china_fundamentals/ifind_inv.xlsx'
wb = excel.Workbooks.Open(outfile)
excel.CalculateUntilAsyncQueriesDone()
wb.Save()
wb.Close()
excel.Quit()


In [10]:
xlapp = win32.gencache.EnsureDispatch("Excel.Application")
xlapp.Interactive = False
xlapp.Application.EnableEvents = False
xlapp.DisplayAlerts = False  # Suppress any Alert windows, which require User action
xlapp.AskToUpdateLinks = True # Disable automatic update linking
xlapp.Visible = True  # Run the Application in the background

outfile = 'C:/Users/harvey/OneDrive/Documents/china_fundamentals/ifind_inv.xlsx'
wb_7 = xlapp.Workbooks.Open(outfile)
wb_7.RefreshAll()
xlapp.CalculateUntilAsyncQueriesDone()


In [ ]:
outfile = 'C:/Users/harvey/OneDrive/Documents/china_fundamentals/ifind_inv.xlsx'
pd.read_excel(outfile, index_col=None, skiprows=[0, 1, 3, 4, 5, 6, 7],sheet_name='base_inv1')

In [ ]:
xlapp.Interactive = True
xlapp.DisplayAlerts = True  # Suppress any Alert windows, which require User action
xlapp.AskToUpdateLinks = True  # Disable automatic update linking
xlapp.Application.EnableEvents = True
xlapp.Visible = True  # Run the Application in the background
xlapp.Quit()    # Close Excel


In [5]:
import openpyxl
import time

In [7]:

# Open the Excel file
workbook = openpyxl.load_workbook('C:/Users/harvey/OneDrive/Documents/china_fundamentals/ifind_inv.xlsx')

time.sleep(10)
# Select the sheet
for sheet_name in ['ferrous_inv1', 'ferrous_inv2', 'base_inv1', 'base_inv2']:
    sheet = workbook[sheet_name]
    sheet.calculate_dimension()
    time.sleep(10)
#     # Refresh formulas in cells A1 to A10
#     sheet.refresh_all_formulas(ranges=['A1:A10'])
    

# Save the changes
workbook.save('ifind_inv.xlsx')


# check upcoming nearby rolling 

In [ ]:
end_date = datetime.date(2023, 1, 3) # datetime.date.today()
start_date = misc.day_shift(end_date, '-120d')
print(start_date, end_date)

roll_dict = {
    'm': 30,
    'RM': 30,
    'rb': 30,
    'ru': 30,
    'sc': 20,
    'lu': 45,
    'eb': 20,
    'eg': 30,
    'cu': 25,
    'jd': 40,
    'lh': 40,
    'T': 20,
    'pg': 30,
}

shift_scope = 15
for prod in ['m', 'RM', 'rb', 'sc', 'lu', 'eb', 'eg', 'cu', 'jd', 'lh', 'T','pg']:
    test_roll = roll_dict[prod] + shift_scope
    nc_df1 = misc.nearby(prod, 1, start_date = start_date, end_date = end_date, roll_rule = f'-{test_roll}b',shift_mode=1)    
    curr_cont = nc_df1['contract'][-shift_scope-1]
    for i in range(shift_scope, 0, -1):
        if curr_cont != nc_df1['contract'][-i]:
            print('%s roll into %s in %s bzdays for current roll rule (%s)' % 
                  (curr_cont, nc_df1['contract'][-i], shift_scope - i, f'-{roll_dict[prod]}b'))
            #print(nc_df1.tail(10))
            break


In [ ]:
#df = pd.read_csv('C:\\dev\\pyktrader3\\process\\option_test\\volgrids_IO_220715.csv', header=None)
#df

In [ ]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

# check contract by product (for OI/volume evolution)

In [ ]:
print(load_fut_by_product('zn', 'SHFE', datetime.date(2022,12,30),datetime.date(2023,1,3)))

# AKShare load future daily data

In [ ]:
from misc_scripts.aks_data_update import update_hist_fut_daily, update_spot_daily, \
                            update_exch_receipt_table, update_exch_inv_table, update_rank_table

In [ ]:
update_hist_fut_daily(datetime.date(2022,12,3), datetime.date(2023,1,3), exchanges = ['GFEX'], flavor = 'mysql', fut_table = 'fut_daily')

In [ ]:
import akshare as ak

In [ ]:
ak.get_futures_daily(start_date = "20230103", end_date = "20230103", market = "SHFE")

# ak.get_futures_daily(start_date='20230103',end_date='20230106', market='DCE')

# check nearby rolling

In [ ]:
start_date = datetime.date(2020,1,2)
end_date = datetime.date(2022,10,14)
nc_df1 = misc.nearby('pg', 1, start_date = start_date, end_date = end_date, roll_rule='-32b',shift_mode=1)
#nc_df2 = misc.nearby('pg', 1, start_date = start_date, end_date = end_date, roll_rule='-30b',shift_mode=1)

nc_df1 #, nc_df2
#nc1_df = misc.nearby('al', 2, start_date = start_date, end_date = end_date, roll_rule='-20b',shift_mode=1)
#nc2_df = misc.nearby('al', 2, start_date = start_date, end_date = end_date, roll_rule='-20b',shift_mode=2)
#print(nc_df, nc1_df, nc2_df)

In [ ]:
start_date = datetime.date(2021,1,1)
end_date = datetime.date(2022,6,28)

#load_fut_by_product('i', 'DCE', start_date ,end_date, freq = 'd')

prodcode = 'sc'
df1 = dataseries.nearby(prodcode, n = 1, start_date = start_date, end_date = end_date,
           roll_rule = '-25b', freq = 'd', shift_mode = 1, 
           adj_field = 'close', calc_fields = ['open', 'close', 'high', 'low'], 
           fill_cont = False,
          )

df2 = misc.nearby(prodcode, n = 1, start_date = start_date, end_date = end_date, roll_rule = '-25b', freq = 'd', shift_mode = 1)

print(df1, df2)

# this is testing for new nearby rolling

In [ ]:
from pycmqlib3.utility.dataseries import *

ferrous_products_mkts = ['rb', 'hc', 'i', 'j', 'jm']
ferrous_mixed_mkts = ['ru', 'FG', 'ZC', 'SM', "SF", 'nr']
base_metal_mkts = ['cu', 'al', 'zn', 'pb', 'ni', 'sn', 'ss']
precious_metal_mkts = ['au', 'ag']
ind_metal_mkts = ferrous_products_mkts + ferrous_mixed_mkts + base_metal_mkts  
petro_chem_mkts = ['l', 'pp', 'v', 'TA', 'MA', 'bu', 'sc', 'fu', 'eg', 'eb', 'lu', 'pg', 'PF'] 
ind_all_mkts = ind_metal_mkts + petro_chem_mkts
ags_oil_mkts = ['m', 'RM', 'y', 'p', 'OI', 'a', 'c', 'cs', 'b'] #, 'b']
ags_soft_mkts = ['CF', 'SR', 'jd', 'AP', 'sp', 'CJ', 'UR', 'SA', 'lh', 'PK',] # 'CY',] 
ags_all_mkts = ags_oil_mkts + ags_soft_mkts
eq_fut_mkts = ['IF', 'IH', 'IC', 'IM',]
bond_fut_mkts = ['T', 'TF', 'TS']
fin_all_mkts = eq_fut_mkts + bond_fut_mkts
commod_all_mkts = ind_all_mkts + ags_all_mkts + precious_metal_mkts
all_markets = commod_all_mkts + fin_all_mkts

In [ ]:
setting_dict = {
    'rb': [datetime.date(2009,8,13), datetime.date(2009,8,13), 5000, '-20b'], 
    'hc': [datetime.date(2014,3,21), datetime.date(2015,12,11), 5000, '-20b'],
    'i': [datetime.date(2013,10,18), datetime.date(2014,4,10), 5000, '-20b'],
    'j': [datetime.date(2011,4,15), datetime.date(2012,5,7), 5000, '-20b'],
    'jm': [datetime.date(2013,3,22), datetime.date(2013,3,22), 5000, '-20b'],
    'ru': [datetime.date(2008,10,10), datetime.date(2008,10,10), 5000, '-20b'], 
    'FG': [datetime.date(2012,12,7), datetime.date(2012,12,7), 5000, '-20b'], 
    'ZC': [datetime.date(2013,9,26), datetime.date(2013,9,26), 5000, '-20b'],
    'SM': [datetime.date(2016,7,26), datetime.date(2016,10,11), 5000, '-20b'], 
    'SF': [datetime.date(2016,7,27), datetime.date(2017,3,13), 5000, '-20b'],    
    'nr': [datetime.date(2019,8,12), datetime.date(2019,8,12), 5000, '-20b'],
    'cu': [datetime.date(2008,1,2), datetime.date(2008,1,2), 5000, '-20b'], 
    'al': [datetime.date(2008,1,2), datetime.date(2008,1,2), 5000, '-20b'],
    'zn': [datetime.date(2008,1,2), datetime.date(2008,1,2), 5000, '-20b'],
    'pb': [datetime.date(2014,7,25), datetime.date(2014,7,25), 5000, '-20b'],
    'ni': [datetime.date(2015,6,17), datetime.date(2015,6,17), 5000, '-20b'], 
    'sn': [datetime.date(2017,5,2), datetime.date(2017,5,2), 5000, '-20b'],
    'ss': [datetime.date(2019,9,25), datetime.date(2020, 1, 6), 5000, '-20b'],
    
    'l': [datetime.date(2008,1,18), datetime.date(2010,4,14), 5000, '-20b'],
    'pp': [datetime.date(2014,2,28), datetime.date(2014,2,28), 5000, '-20b'], 
    'v': [datetime.date(2009,5,25), datetime.date(2010,8,16), 5000, '-20b'],   
    'TA': [datetime.date(2007,7,10), datetime.date(2008,7,10), 5000, '-20b'],
    'MA': [datetime.date(2011,10,28), datetime.date(2013,10,22), 5000, '-20b'], 
    'sc': [datetime.date(2018, 3, 26), datetime.date(2018,8,30), 5000, '-20b'],
    'bu': [datetime.date(2015, 1,16), datetime.date(2015,7,1), 5000, '-20b'],
    'fu': [datetime.date(2018, 7, 17), datetime.date(2018,10,10), 5000, '-20b'],
    'lu': [datetime.date(2020, 10, 9), datetime.date(2020, 10, 9), 5000, '-20b'], 
    'eb': [datetime.date(2020, 5, 6), datetime.date(2020, 6, 1), 5000, '-20b'],
    'eg': [datetime.date(2019, 4, 2), datetime.date(2019, 4, 2), 5000, '-20b'],
    'pg': [datetime.date(2020, 9, 7), datetime.date(2020, 9, 7), 5000, '-20b'],
    'PF': [datetime.date(2021,1,5), datetime.date(2021,3,1), 5000, '-20b'],
        
    'a': [datetime.date(2008,1,2), datetime.date(2008,1,2), 5000, '-20b'],
    'b': [datetime.date(2019,7,1), datetime.date(2019,7,1), 5000, '-20b'],
    'm': [datetime.date(2008,1,2), datetime.date(2008,1,2), 5000, '-20b'],
    'RM': [datetime.date(2012,12,28), datetime.date(2012,12,28), 5000, '-20b'], 
    'y': [datetime.date(2008,1,2), datetime.date(2008,1,2), 5000, '-20b'],  
    'p':  [datetime.date(2008,1,2), datetime.date(2008,1,2), 4000, '-20b'], 
    'OI': [datetime.date(2013,2,21), datetime.date(2013,2,21), 5000, '-20b'], 
    'c': [datetime.date(2008,1,2), datetime.date(2008,1,2), 5000, '-20b'],
    'cs': [datetime.date(2014,12,19), datetime.date(2014,12,19), 5000, '-20b'], 
    
    'UR': [datetime.date(2019, 8, 9), datetime.date(2019, 8, 9), 5000, '-20b'],
    'SA': [datetime.date(2019,12,6), datetime.date(2019,12,6), 5000, '-20b'],
    'CF': [datetime.date(2008,7,14), datetime.date(2008,7,14), 5000, '-20b'], 
    'CY': [datetime.date(2017, 9, 1), datetime.date(2017, 9, 1), 5000, '-20b'],
    'SR': [datetime.date(2008,7,10), datetime.date(2008,7,10), 5000, '-20b'],
    'AP': [datetime.date(2017,12,22), datetime.date(2017,12,22), 5000, '-20b'],
    'jd': [datetime.date(2013,11,8), datetime.date(2013,11,8), 5000, '-20b'], 
    'lh': [datetime.date(2021,1,8), datetime.date(2021,1,8), 5000, '-20b'],    
    'PK': [datetime.date(2021,2,1), datetime.date(2021,2,1), 5000, '-20b'],
    'CJ': [datetime.date(2019,4,30), datetime.date(2019,4,30), 5000, '-20b'],
    'rr': [datetime.date(2019, 9, 1), datetime.date(2019, 9, 1), 5000, '-20b'],
    'sp': [datetime.date(2018,11,27), datetime.date(2018,11,27), 5000, '-20b'],
    
    'au': [datetime.date(2010,1,4), datetime.date(2015,1,16), 5000, '-20b'],
    'ag': [datetime.date(2012,5,10), datetime.date(2019,6,26), 5000, '-20b'],
    'IF': [datetime.date(2010,5,1), datetime.date(2010,5,1), 5000, '-20b'], 
    'IH': [datetime.date(2015,5,1), datetime.date(2015,5,1), 5000, '-20b'], 
    'IC': [datetime.date(2015,5,1), datetime.date(2015,5,1), 5000, '-20b'], 
    'TF': [datetime.date(2019,6,1), datetime.date(2019,6,1), 5000, '-20b'],
    'T': [datetime.date(2019,4,1), datetime.date(2019,4,1), 5000, '-20b'], 
    'TS': [datetime.date(2018, 9, 1), datetime.date(2018, 9, 1), 5000, '-20b'],
    'PM': [datetime.date(2013,10,1), datetime.date(2013,10,1), 5000, '-20b'], 
    'RI': [datetime.date(2013,1,1), datetime.date(2013,1,1), 5000, '-20b'], 
    'WH': [datetime.date(2014,5,1), datetime.date(2014,5,1), 5000, '-20b'], 
}

# script to batch generate nearby rolling csv file

In [ ]:
start_date = datetime.date(2008,1,1)
end_date = datetime.date(2022,7,22)
freq = 'd'
roll_mode = 0
nb_cont = 2

#sn need to fix 2019-12
# IF IC IH need to roll quarterly and how to roll monthly
roll_name = 'nroll'
roll_win = 3
cont_thres = 50_000
cont_ratio = [1.0, 0.0]
folder = "C:\\dev\\pyktrader3\\data\\carry"
for prodcode in commod_all_mkts: #bond_fut_mkts: #: # petro_chem_mkts
    min_thres = 7_500
    roll_cutoff = '-20b'
    contract_filter = None
    if prodcode in ['IF', 'IC', 'IH', 'sc',]:
        min_thres = 0.1 * min_thres
    elif prodcode in ['j', 'ni', 'sn', 'cu', 'bc', 'lh']:
        min_thres = 0.2 * min_thres
    elif prodcode in ['ru', 'nr', 'al', 'zn', 'pb', 'jm', 'ss', 'PK', 'b',]:
        min_thres = 0.5 * min_thres
    elif prodcode in ['SM', 'SF', 'FG', 'i', 'AP', 'eb', ]:
        min_thres = 0.8 * min_thres
    elif prodcode in ['T', 'TF', 'TS']:
        min_thres = 0
    if prodcode in ['IF', 'IC', 'IH']:
        roll_cutoff = '0b'

#     if prodcode in ['j', ]:
#         contract_filter = prod_main_cont_filter
    roll_kwargs = {'roll_win': roll_win, 'roll_cutoff': roll_cutoff, 'cont_ratio': cont_ratio,
                   'contract_filter': contract_filter, 'min_thres': min_thres}
    df = load_processed_fut_by_product(prodcode, start_date=start_date, end_date=end_date, freq=freq, **roll_kwargs)
    roll_map, daily_cont = rolling_fut_cont(df, nb_cont=nb_cont, cont_thres=cont_thres, roll_mode=roll_mode)
    if roll_map.iloc[-1, 1] == None:
        roll_map.iat[-1, 1] = misc.default_next_main_contract(roll_map.iloc[-1, 0], start_date, end_date)
    flag = roll_map[1].isna()
    print(prodcode, roll_map)
    roll_map.loc[flag, 1] = roll_map.loc[flag.shift(1).fillna(False), 0].values
    file_name = "%s\\%s_%s.csv" % (folder, prodcode, roll_name) 
    roll_map.to_csv(file_name)



# single product rolling testing

In [ ]:
start_date = datetime.date(2010,1,1)
end_date = datetime.date(2022,7,15)
freq = 'd'
roll_mode = 0
nb_cont = 2

roll_name = 'nroll'
roll_win = 3
cont_thres = 50_000
cont_ratio = [1.0, 0.0]

prodcode = 'rb'
min_thres = 7_500
roll_cutoff = '-20b'
contract_filter = None

roll_kwargs = {'roll_win': roll_win, 'roll_cutoff': roll_cutoff, 'cont_ratio': cont_ratio,
               'contract_filter': contract_filter, 'min_thres': min_thres}
df = load_processed_fut_by_product(prodcode, start_date=start_date, end_date=end_date, freq=freq, **roll_kwargs)
roll_map, daily_cont = rolling_fut_cont(df, nb_cont=nb_cont, cont_thres=cont_thres, roll_mode=roll_mode)

In [ ]:
end_date = datetime.date(2022,9,22)
shift_mode = 1

roll_mode = 0
roll_name = 'nroll'
nb_cont = 2

roll_win = 3
cont_thres = 50_000
cont_ratio = [1.0, 0.0]

min_thres = 0
roll_cutoff = '-5b'
contract_filter = None
    
folder = "C:\\dev\\pyktrader3\\data\\carry"

data_df = pd.DataFrame()
for prodcode in ['rb', 'hc', 'i', 'j']: 
    if prodcode in ['IF', 'IC', 'IH', 'sc',]:
        min_thres = 0.1 * min_thres
    elif prodcode in ['j', 'ni', 'sn', 'cu', 'bc', 'lh']:
        min_thres = 0.2 * min_thres
    elif prodcode in ['ru', 'nr', 'al', 'zn', 'pb', 'jm', 'ss', 'PK', 'b',]:
        min_thres = 0.5 * min_thres
    elif prodcode in ['SM', 'SF', 'FG', 'i', 'AP', 'eb', ]:
        min_thres = 0.8 * min_thres
    elif prodcode in ['T', 'TF', 'TS']:
        min_thres = 0
    if prodcode in ['IF', 'IC', 'IH']:
        roll_cutoff = '0b'
    roll_kwargs = {'roll_win': roll_win, 'roll_cutoff': roll_cutoff, 'cont_ratio': cont_ratio,
                   'contract_filter': contract_filter, 'min_thres': min_thres}
    
    setting = setting_dict[prodcode]
    file_name = "%s\\%s_%s.csv" % (folder, prodcode, roll_name) 
    roll_map = pd.read_csv(file_name).set_index('date')
    nb_df = nearby_series(prodcode, start_date=setting[0], end_date = end_date, shift_mode = shift_mode,
                  freq='d', roll_kwargs=roll_kwargs, roll_map = roll_map)
    for key in nb_df:
        fields = nb_df[key].columns
        nb_df[key]['product'] = prodcode
        nb_df[key]['code'] = key
        data_df = data_df.append(nb_df[key])
print(fields)    
#     df = pd.concat([nb_df[key] for key in nb_df],axis=1)
#     df.to_csv("%s\\combined_%s_%s.csv" % (folder, prodcode, roll_name))

In [ ]:
df = pd.pivot_table(data_df.reset_index(), index='date', columns = ['product', 'code'], values = list(fields), aggfunc = 'last')
df = df.reorder_levels([1, 2, 0], axis=1).sort_index(axis=1)
df.columns.rename(['field', 'product', 'code'], inplace = True)
print(df)

In [ ]:

df[df.columns[(df.columns.get_level_values(2) == 'close') & (df.columns.get_level_values(1) == 'c0')]].droplevel([1, 2],axis=1)

In [1]:
import math
math.log(2)

0.6931471805599453

In [3]:
import numpy as np
np.log(2)

0.6931471805599453

In [ ]:
prodcode = 'i'

start_date = datetime.date(2014,1,1)
end_date = datetime.date(2022,6,30)

freq = 'd'
shift_mode = 1
contract_filter = None # prod_main_cont_exch
roll_mode = 0
nb_cont = 2

roll_win = 3
roll_cutoff = '-20b'
min_thres = 5_000
cont_thres = 40_000
cont_ratio = [1.0, 0.0]

nb_df= nearby_series(prodcode, start_date=start_date, end_date=end_date, shift_mode=shift_mode,
                     nb_cont=nb_cont, cont_thres=cont_thres, roll_mode=roll_mode,freq=freq, 
                     roll_kwargs={'roll_win': roll_win, 'roll_cutoff': roll_cutoff, 'cont_ratio': cont_ratio,
                                  'contract_filter': contract_filter, 'min_thres': min_thres})
print(nb_df)

# this is for backtest metric testing

In [ ]:
start_date = datetime.date(2018, 1, 2)
end_date = datetime.date(2022,7,18)
shift_mode = 1

roll_mode = 0
roll_name = 'nroll'
nb_cont = 2

prodcode = 'rb'

roll_kwargs = {'roll_win': 3, 'roll_cutoff': '-5b', 'cont_ratio': [1.0, 0.0],
               'contract_filter': None, 'min_thres': 0}

folder = "C:\\dev\\pyktrader3\\data\\carry"
file_name = "%s\\%s_%s.csv" % (folder, prodcode, roll_name) 
roll_map = pd.read_csv(file_name).set_index('date')

nb_df = nearby_series(prodcode, start_date=start_date, end_date = end_date, shift_mode = shift_mode,
              freq='d', roll_kwargs=roll_kwargs, roll_map = roll_map)
print(nb_df)

In [ ]:
xdf = nb_df['c0']

xdf['holding'] = (xdf['price_chg'] - xdf['price_chg'].rolling(14).mean()).shift(1).fillna(0)
xdf

In [ ]:
metrics = MetricsBase(xdf['holding'].to_frame('rb'), xdf['price_chg'].to_frame('rb'))


In [ ]:
portfolio_pnl = metrics._lagged_portfolio_pnl().to_frame(name='total')
portfolio_pnl['Month'] = portfolio_pnl.index.month
portfolio_pnl.index = [i.replace(month=1) for i in portfolio_pnl.index]
seasonal_pnl = portfolio_pnl.pivot(columns = 'Month', values = 'total').fillna(0.0)
print(seasonal_pnl)

In [ ]:
#seasonal_pnl.plot(x=seasonal_pnl.index.astype(str))
seasonal_pnl.set_index(seasonal_pnl.index.astype('str')).plot(rot=30)

In [ ]:
#metrics.holdings, metrics.returns
# res = metrics.seasonal_pnl()
res_df = metrics.lead_lag()
print(res_df)

In [ ]:
ts = res_df['leadlag_sharpes'].loc['fullsample']
ax = ts.plot(kind='bar',figsize=(6, 6))
new_ticks = np.linspace(-20, 60, 17)
ax.set_xticks(np.interp(new_ticks, ts.index, np.arange(ts.size)))
ax.set_xticklabels(new_ticks)


In [ ]:
res = metrics.lagged_pnl(lags=[1,2,5,10,20])
res['cumpnl'].plot()
print(res['sharpe'])

In [ ]:
res = metrics.smoothed_pnl(smooth_hls=[1, 2, 5, 10, 20])
res['cumpnl'].plot()
print(res['sharpe'])


In [ ]:
nb_df['holding'].to_frame('al').plot()

In [ ]:
pd.to_datetime('2021-01-04').replace(day=1)

In [ ]:
calendar_aggregation(nb_df, period = 'weekly', how = 'sum')

In [ ]:
start_date = datetime.date(2014,1,1)
end_date = datetime.date(2022,6,17)

nb_df = dataseries.nearby('al', 1, start_date = start_date, end_date = end_date, roll_rule='-30b',shift_mode=1)
#nb1_df = dataseries.nearby('sc', 2, start_date = start_date, end_date = end_date, roll_rule='-20b',shift_mode=1)
#nb2_df = nearby('al', 2, start_date = start_date, end_date = end_date, roll_rule='-30b',shift_mode=2)
print(nb_df)

In [ ]:
print(load_fut_by_product('eg', 'DCE', datetime.date(2022,5,20) ,datetime.date(2022,5,20)))
#print(load_fut_by_product('sc', 'INE', datetime.date(2022,3,2) ,datetime.date(2022,3,2)))

In [ ]:
#nb_df[nb1_df['instID'] == nb_df['instID']]
#nb_df.close.plot(color='r')

#nb1_df.close.plot(color='y')
print(nb1_df.close.diff().max(), nb1_df.price_chg.max())
#nb_df[['close']].plot()
#flag = nb1_df.close.diff()>1.0
#nb1_df[flag | flag.shift(-1)]
#nb1_df.loc[datetime.date(2018,9,8):datetime.date(2018,9,20):]

In [ ]:
start_date = datetime.date(2011,1,1)
end_date = datetime.date(2022, 1, 28)
prodcode = 'ZC'
roll_rule = '-30b'
shift_mode = 1
n = 1

adj_field = 'close'
calc_fields = ['open', 'close', 'high', 'low']
contract_filter = [1, 5, 9]

exch = misc.prod2exch(prodcode)
xdf = load_fut_by_product(prodcode, exch, start_date ,end_date)
xdf['expiry'] = xdf['instID'].apply(lambda x: misc.contract_expiry(x, hols = misc.CHN_Holidays))
xdf['month'] = xdf['instID'].apply(lambda x: misc.inst2contmth(x)%100)
xdf = xdf.sort_values(['instID', 'date'])

if shift_mode == 2:
    xdf['price_chg'] = np.log(xdf[adj_field]).diff()
else:
    xdf['price_chg'] = xdf[adj_field].diff()        
xdf.loc[xdf['instID']!=xdf['instID'].shift(1), 'price_chg'] = 0
if (roll_rule[0] == '-') and (roll_rule[-1] in ['b', 'd']):
    xdf['roll_date'] = xdf['expiry'].apply(lambda x: misc.day_shift(x, roll_rule, hols = misc.CHN_Holidays))
    xdf = xdf[xdf.date <= xdf['roll_date']]
else:
    xdf['roll_date'] = xdf['expiry']
if contract_filter:
    xdf = xdf[xdf.month.isin(contract_filter)]
df = pd.pivot_table(xdf, index = 'date', columns = 'expiry', values = 'instID', aggfunc = 'first')
df1 = df.apply(lambda x: pd.Series(x.dropna().values), axis=1)
df1 = df1.reset_index()

col_df = df1[['date', n-1]].rename(columns = {n-1: 'instID'})
#if len(col_df[col_df['instID'].isna()])> 0:
#    raise ValueError('There are nan values for product=%s, nearby=%s, roll=%s' % (prodcode, str(n), roll_rule))
col_df = col_df.fillna(method = 'ffill')
out_df = pd.merge(col_df, xdf,left_on = ['date', 'instID'], right_on=['date', 'instID'], how = 'left')

if shift_mode > 0:
    cum_adj = out_df.loc[::-1, 'price_chg'].cumsum().shift(1).fillna(0)[::-1]
    if shift_mode == 2:
        adj_price = out_df[adj_field].iloc[-1]/np.exp(cum_adj)
        out_df['shift'] = np.log(adj_price) - np.log(out_df[adj_field])
    else:
        adj_price = out_df[adj_field].iloc[-1] - cum_adj
        out_df['shift'] = adj_price - out_df[adj_field]    

    for cfield in calc_fields:
        if shift_mode == 2:
            out_df[cfield] = out_df[cfield] * np.exp(out_df['shift'])
        else:
            out_df[cfield] = out_df[cfield] + out_df['shift']
else:
    out_df['shift'] = 0
out_df = out_df.set_index('date')

In [ ]:
load_fut_by_product('TA', 'CZCE', start_date ,end_date)

In [ ]:
#out_df[out_df.instID.isna()]

#out_df.loc[datetime.date(2017,11,20):datetime.date(2017,12,10),:]
#out_df.close.plot()
print(out_df.close.diff().max(),out_df.price_chg.max())

In [ ]:
price_adj = out_df.loc[::-1, 'price_chg'].cumsum().shift(1).fillna(0)[::-1]
price_close = out_df['close'].iloc[-1] - price_adj
price_shift = price_close - out_df['close']
print(price_close, price_shift)

In [ ]:
xdf = xdf.sort_values(['instID', 'date'])
xdf['chg'] = xdf['close'].diff()
flag = xdf['instID']!=xdf['instID'].shift(1)
xdf.loc[flag, 'chg'] = 0
print(xdf)

In [ ]:
adf = df.iloc[:, df.columns.get_level_values(0)=='close']
nb_df = nearby(adf, roll = 30)
print(nb_df[0])

In [ ]:
import pypfopt

In [ ]:
def nearby(prodcode, n = 1, start_date = None, end_date = None, 
           roll_rule = '-20b', shift_mode = 0, 
           adj_field = 'close', calc_fields = ['open', 'close', 'high', 'low'], 
           contract_filter = None, fill_cont = False,
          ):
    exch = misc.prod2exch(prodcode)
    xdf = load_fut_by_product(prodcode, exch, start_date ,end_date)
    xdf['expiry'] = xdf['instID'].apply(lambda x: misc.contract_expiry(x, hols = misc.CHN_Holidays))
    xdf['month'] = xdf['instID'].apply(lambda x: misc.inst2contmth(x)%100)
    xdf = xdf.sort_values(['instID', 'date'])

    if shift_mode == 2:
        xdf['price_chg'] = np.log(xdf[adj_field]).diff()
    else:
        xdf['price_chg'] = xdf[adj_field].diff()        
    xdf.loc[xdf['instID']!=xdf['instID'].shift(1), 'price_chg'] = 0
    if (roll_rule[0] == '-') and (roll_rule[-1] in ['b', 'd']):
        xdf['roll_date'] = xdf['expiry'].apply(lambda x: misc.day_shift(x, roll_rule, hols = misc.CHN_Holidays))
        xdf = xdf[xdf.date <= xdf['roll_date']]
    else:
        xdf['roll_date'] = xdf['expiry']
    if contract_filter:
        xdf = xdf[xdf.month.isin(contract_filter)]
    df = pd.pivot_table(xdf, index = 'date', columns = 'expiry', values = 'instID', aggfunc = 'first')
    df1 = df.apply(lambda x: pd.Series(x.dropna().values), axis=1)
    df1 = df1.reset_index()
    col_df = df1[['date', n-1]].rename(columns = {n-1: 'instID'})
    if len(col_df[col_df['instID'].isna()])> 0:
        if fill_cont:
            col_df = col_df.fillna(method = 'ffill')
        else:
            raise ValueError('There are nan values for product=%s, nearby=%s, roll=%s' % (prodcode, str(n), roll_rule))
    out_df = pd.merge(col_df, xdf,left_on = ['date', 'instID'], right_on=['date', 'instID'], how = 'left')
    if shift_mode > 0:
        cum_adj = out_df.loc[::-1, 'price_chg'].cumsum().shift(1).fillna(0)[::-1]
        if shift_mode == 2:
            adj_price = out_df[adj_field].iloc[-1]/np.exp(cum_adj)
            out_df['shift'] = np.log(adj_price) - np.log(out_df[adj_field])
        else:
            adj_price = out_df[adj_field].iloc[-1] - cum_adj
            out_df['shift'] = adj_price - out_df[adj_field]    

        for cfield in calc_fields:
            if shift_mode == 2:
                out_df[cfield] = out_df[cfield] * np.exp(out_df['shift'])
            else:
                out_df[cfield] = out_df[cfield] + out_df['shift']
    else:
        out_df['shift'] = 0
    out_df = out_df.set_index('date')
    return out_df

In [ ]:
import json

with open("C:/dev/wtdev/common/hots.json", 'r') as infile:
    config = json.load(infile)
print(config)

In [ ]:

pd.DataFrame.from_records(config['CFFEX']['IF'])